# <center> Исследование данных о зарплатах в сфере Data Science

# Постановка задачи

Произвести исследование данных HR агентсва за период 2020-2022 годы. На основе полученных данных произвести анализ и ответить на следующие вопросы: 

- Факторы, влияющие на уровень зарплаты специалиста в области Data Science 
- Наблюдается ли ежегодный рост зарплат у специалистов Data Scientist?
- Как соотносятся зарплаты Data Scientist и Data Engineer в 2022 году?
- Как соотносятся зарплаты специалистов Data Scientist в компаниях различных размеров?
- Есть ли связь между наличием должностей Data Scientist и Data Engineer и размером компании?

В ходе исследования принят уровень значимости $\alpha=0.05$.

**Импортируем необходимые в ходе исследования библиотеки, задаем коэфициент уровня значимости.**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
import scipy.stats as stats
from scipy import stats
import statistics
import country_converter as coco

alpha = 0.05

# Чтение и первичное изучение данных. 

**Произведем предварительную обработку данных (наличие пропусков, дубликтов, неинформативных признаков).**

In [40]:
# Произведем чтение данных и выведем первые 5 строк для изучения

data = pd.read_csv('data/ds_salaries_SF.csv', index_col = 'Unnamed: 0')

display(data.head())

print('Имеется {} строк данных, и {} признаков'.
      format(data.shape[0], data.shape[1]))

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


Имеется 607 строк данных, и 11 признаков


Данные содеражат следующие столбцы: 

- **work_year** - Год, в котором была выплачена зарплата.
- **experience_level** -	Опыт работы на этой должности в течение года со следующими возможными значениями:
    * EN — Entry-level/Junior;
    * MI — Mid-level/Intermediate;
    * SE — Senior-level/Expert;
    * EX — Executive-level/Director.
- **employment_type** -	Тип трудоустройства для этой роли:
    * PT — неполный рабочий день;
    * FT — полный рабочий день;
    * CT — контракт;
    * FL — фриланс.
- **job_title** -	Роль, в которой соискатель работал в течение года.
- **salary** -	Общая выплаченная валовая сумма заработной платы.
- **salary_currency** -	Валюта выплачиваемой заработной платы в виде кода валюты ISO 4217.
 - **salary_in_usd** -	Зарплата в долларах США (валютный курс, делённый на среднее значение курса доллара США за соответствующий год через fxdata.foorilla.com).
- **employee_residence** -	Основная страна проживания сотрудника в течение рабочего года в виде кода страны ISO 3166.
- **remote_ratio** -	Общий объём работы, выполняемой удалённо. Возможные значения:
    * 0 — удалённой работы нет (менее 20 %);
    * 50 — частично удалённая работа;
    * 100 — полностью удалённая работа (более 80 %).
- **company_location** -	Страна главного офиса работодателя или филиала по контракту в виде кода страны ISO 3166.
- **company_size** -	Среднее количество людей, работавших в компании в течение года:
    * S — менее 50 сотрудников (небольшая компания);
    * M — от 50 до 250 сотрудников (средняя компания);
    * L — более 250 сотрудников (крупная компания).

In [41]:
# Произведем проверку наличия пропусков 

null_cols = data.isnull().sum()
data_null_cols = null_cols[null_cols>0]

if data_null_cols.shape[0] != 0:
    print('Имеются пропуски, нужна обработка')
else:
    print('Пропусков нет')


# Произведем поиск и удаление дубликатов (при наличии) 
    
data_dupl = data[data.duplicated()]

if data_dupl.shape[0] != 0:
    data = data.drop_duplicates()
    print('Имелись дубликаты в количестве {} шт. ' 
        'Данные очищены от дубликатов.'.format(data_dupl.shape[0]))

else:
    print('Дубликатов нет')

Пропусков нет
Имелись дубликаты в количестве 42 шт. Данные очищены от дубликатов.


In [42]:
# Произведем очистку данных от неинформативных признаков

data = data.drop('salary', axis=1)
data = data.drop('salary_currency', axis=1)


#### Описательный анализ данных

Перейдем теперь к структуре DataFrame, ознакомимся с типами представленных данных, выведем очищенный DataFrame для анализа возможных методов преобразования данных или создания новых признаков

In [43]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 565 entries, 0 to 606
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           565 non-null    int64 
 1   experience_level    565 non-null    object
 2   employment_type     565 non-null    object
 3   job_title           565 non-null    object
 4   salary_in_usd       565 non-null    int64 
 5   employee_residence  565 non-null    object
 6   remote_ratio        565 non-null    int64 
 7   company_location    565 non-null    object
 8   company_size        565 non-null    object
dtypes: int64(3), object(6)
memory usage: 44.1+ KB


Выведем статистическую информацию по признакам категории object:

In [44]:
data.describe(include='object')

,experience_level,employment_type,job_title,employee_residence,company_location,company_size
count,565,565,565,565,565,565
unique,4,4,50,57,50,3
top,SE,FT,Data Scientist,US,US,M
freq,243,546,130,295,318,290


Мы видим, что часть признаков, таких как job_title, employee_residence, company_location требуют дальнешей обработки, т.к. количество уникальных значений слишком велико.


Выведем статистическую информацию по признакам категории int64:


In [45]:
data.describe(include='int64')

,work_year,salary_in_usd,remote_ratio
count,565.000000,565.000000,565.000000
mean,2021.364602,110610.343363,69.911504
std,0.698138,72280.702792,40.900666
min,2020.000000,2859.000000,0.000000
25%,2021.000000,60757.000000,50.000000
50%,2021.000000,100000.000000,100.000000
75%,2022.000000,150000.000000,100.000000
max,2022.000000,600000.000000,100.000000


Статистическая информация по таким признакам как work_year и remote_ratio в данном случае не представляет особого интереса. Далее в исследовании тип данных будет изменен. 

Статистическая информавция по признаку заработной платы дает нам понимание медианной и средней заработных плат, 100.000 и 110.610 USD соответственно. Диапазон между первым и третьим квинтилями - от 60.757 до 150.000 USD.

Изучим признаки **employee_residence** и **company_location** более подробно для формирования понимания того, как лучше их преобразовать в дальнейшем. 

In [46]:
display(data['employee_residence'].value_counts(normalize=True)[0:7])
display(data['company_location'].value_counts(normalize=True)[0:7])

employee_residence
US    0.522124
GB    0.076106
IN    0.053097
CA    0.047788
DE    0.042478
FR    0.031858
ES    0.026549
Name: proportion, dtype: float64

company_location
US    0.562832
GB    0.081416
CA    0.049558
DE    0.047788
IN    0.042478
FR    0.026549
ES    0.024779
Name: proportion, dtype: float64

В обоих признаках лидером по количеству упоминаний является США - более половины. Расшифровка кодировки стран топ-7: 
* US - США
* GB - Великобритания 
* IN - Индия 
* CA - Канада 
* DE - Германия 
* FR - Франция 
* ES - Испания

# Разведывательный анализ данных

Рассмотрим сначала очевидные признаки, которые отнесем к **категориальным**:

- experience_level
- employment_type
- company_size

Теперь рассмотрим признаки, которые следует так же отнести к **категориальным**, но они требуют дальнейшего преобразования:

- **work_year**:

    В наших данных всего три года, представления этих данных в целочисленном виде не имеет смысла

- **remote_ratio**:

    Аналогично, данный признак говорит нам о том, работает ли сотрудник удаленно, гибридно или офисно, это относится к категориальному признаку

- **job_title**:

    С этим признаком сложнее, имеется 50 уникальных значений при 565 строках. Для анализа такие данные непригодны. С ними поступим следующим образом: разобьём все позиции на три категории: DS - Data Science, DE - Data Engineering, DA - Data Analytic

- **employee_residence**:

    Аналогично с признаком job_title напрашивается необходимость укрупнения приведенных данных. 
    При выборе метода преобразования можно воспользоваться двумя вариантами, вариант первый: преобразовать данные по виду топ-5 + остальные, но в этом случае под "остальные" попадут 20 процентов значений. Поэтому, пойдем другим путем.
    
    Т.к. данные представлены в виде кодировки ISO 3166 alpha_2, мы можем воспользоваться сторонней библиотекой *country_converter* для преобразования страны в регион. Единственное, что стоит сделать вне этой библиотеки - это обратить внимание на страны бывшего СССР, т.к. все-таки развитие стран отлично от западно-европейского, и, возможно, имеется своя специфика. Поэтому, страны бывшего СССР будут фигурировать в дальнейшем как *ex_USSR*. 
    При этом, в дальнейшем мы будем держать в уме, что когда речь идет о значении America - это США в бОльшей части, когда Europe - это Великобритания + Германия + Франция + Испания, а когда Asia - Индия. 

- **company_location**:

    Все рассуждения аналогичны признаку employee_residence.  
      
К **числовым признакам** можно отнести только уровень заработной платы salary_in_usd.



Приступим к преобразованию данных в нашем датасете. 

In [47]:
# Преобразования названия должности в сферу деятельности 

# Функция преобразования описания работы в сферу работы

def job_field(x):
   
    x = x.replace(' ', '')
    x = x.lower()
    
    if 'scien' in x or \
        'machin' in x or \
        'comput' in x or \
        'ml' in x or \
        'cv' in x or \
        'nlp' in x or \
        'ai' in x:

        return 'DS'

    if 'analy' in x:

        return 'DA'

    else:

        return 'DE' 
    
# Применим функцию и переименуем признак 

data['job_title'] = data['job_title'].apply(job_field)

data = data.rename(columns = {'job_title': 'job_field'})


# Преобразования страны в регион 

# Cписок стран бывшего СССР

ex_ussr_list = ['AM', 'AZ', 'BY', 'GE', 'KZ', 
                'KG', 'LV', 'LT', 'MD', 'TJ', 
                'TM', 'UZ', 'UA', 'EE', 'RU']


# Функция преобразования страны в регион

def continent_filling(x):
    
    if x in ex_ussr_list:
        
        return 'ex_USSR'
    
    else:
        converter = coco.CountryConverter()
        result = converter.convert(names = x, src='ISO2', to='continent')
        
        return result

# Применим функцию

data['company_location'] = \
    data['company_location'].apply(continent_filling)
    
data['employee_residence'] = \
    data['employee_residence'].apply(continent_filling)


# Обновим индексы и сохраним DF в новый файл 
# Необходимость сохранения нового файла вызвана долгой обработкой 
# при использовании библиотеки country_converter 
# В дальнейшем целесообразно преобразовывать и кодировать новый DF /


data.reset_index()

data.to_csv('data/ds_salaries_upd.csv', index=False)

# ЗДЕСЬ ВСЕ

In [48]:
data_upd = pd.read_csv('data/ds_salaries_upd.csv')

# Произведем преобразование признаков в категориальные 

for elem in data.columns:
    if elem == 'salary_in_usd':
        pass
    else:
        data['{}'.format(elem)] = data['{}'.format(elem)].astype('category')

data_upd.describe(exclude='int64')
data_upd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 565 entries, 0 to 564
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           565 non-null    int64 
 1   experience_level    565 non-null    object
 2   employment_type     565 non-null    object
 3   job_field           565 non-null    object
 4   salary_in_usd       565 non-null    int64 
 5   employee_residence  565 non-null    object
 6   remote_ratio        565 non-null    int64 
 7   company_location    565 non-null    object
 8   company_size        565 non-null    object
dtypes: int64(3), object(6)
memory usage: 39.9+ KB


In [49]:
jobs = ['Data Scientist', 'Machine Learning Scientist',
       'Big Data Engineer', 'Product Data Analyst',
       'Machine Learning Engineer', 'Data Analyst', 'Lead Data Scientist',
       'Business Data Analyst', 'Lead Data Engineer', 'Lead Data Analyst',
       'Data Engineer', 'Data Science Consultant', 'BI Data Analyst',
       'Director of Data Science', 'Research Scientist',
       'Machine Learning Manager', 'Data Engineering Manager',
       'Machine Learning Infrastructure Engineer', 'ML Engineer',
       'AI Scientist', 'Computer Vision Engineer',
       'Principal Data Scientist', 'Data Science Manager', 'Head of Data',
       '3D Computer Vision Researcher', 'Data Analytics Engineer',
       'Applied Data Scientist', 'Marketing Data Analyst',
       'Cloud Data Engineer', 'Financial Data Analyst',
       'Computer Vision Software Engineer',
       'Director of Data Engineering', 'Data Science Engineer',
       'Principal Data Engineer', 'Machine Learning Developer',
       'Applied Machine Learning Scientist', 'Data Analytics Manager',
       'Head of Data Science', 'Data Specialist', 'Data Architect',
       'Finance Data Analyst', 'Principal Data Analyst',
       'Big Data Architect', 'Staff Data Scientist', 'Analytics Engineer',
       'ETL Developer', 'Head of Machine Learning', 'NLP Engineer',
       'Lead Machine Learning Engineer', 'Data Analytics Lead']

str_1 = 'Big Data Architect'
str_1 = str_1.replace(' ', '')
str_1 = str_1.lower()
str_1



def job_field(x):
   
    x = x.replace(' ', '')
    x = x.lower()
    
    if 'scien' in x or \
        'machin' in x or \
        'comput' in x or \
        'ml' in x or \
        'cv' in x or \
        'nlp' in x or \
        'ai' in x:

        return 'DS'

    if 'analy' in x:

        return 'DA'

    else:

        return 'DE' 


In [50]:
data.describe(include='int64')

,salary_in_usd
count,565.000000
mean,110610.343363
std,72280.702792
min,2859.000000
25%,60757.000000
50%,100000.000000
75%,150000.000000
max,600000.000000


Изучив структруру и типы данных 

In [51]:
data['work_year'] = data['work_year'].astype('category')
data.info()
data.describe(exclude='int64')



<class 'pandas.core.frame.DataFrame'>
Index: 565 entries, 0 to 606
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   work_year           565 non-null    category
 1   experience_level    565 non-null    category
 2   employment_type     565 non-null    category
 3   job_field           565 non-null    category
 4   salary_in_usd       565 non-null    int64   
 5   employee_residence  565 non-null    category
 6   remote_ratio        565 non-null    category
 7   company_location    565 non-null    category
 8   company_size        565 non-null    category
dtypes: category(8), int64(1)
memory usage: 14.6 KB


,work_year,experience_level,employment_type,job_field,employee_residence,remote_ratio,company_location,company_size
count,565,565,565,565,565,565,565,565
unique,3,4,4,3,6,3,6,3
top,2022,SE,FT,DS,America,100,America,M
freq,278,243,546,279,336,346,355,290


In [52]:
print('Количество уникальных значений признаков')
for title in data.columns:
    print(f'{title}: ', data['{}'.format(title)].nunique())

Количество уникальных значений признаков
work_year:  3
experience_level:  4
employment_type:  4
job_field:  3
salary_in_usd:  369
employee_residence:  6
remote_ratio:  3
company_location:  6
company_size:  3


In [53]:
# data['work_year'] = data['work_year'].astype('category')
# data['experience_level'] = data['experience_level'].astype('category')
# data['employment_type'] = data['employment_type'].astype('category')
# data['work_year'] = data['work_year'].astype('category')
# data['work_year'] = data['work_year'].astype('category')



**Промежуточный вывод:**

В ходе предварительной обработки данных мы выяснили, что отсутствуют незаполненные признаки. Так же, обнаружили 42 дубликата и избавились от них. Так же, произвели очистку данных от неинформативных признаков, а именно: значение зарплаты и валюта зарплаты (salary, salary_currency). Причиной этому послужило наличие признака "salary_in_usd", который уже унифицирован для оценки уровня зароботных плат сотрудников и его хватит для решения поставленных задач. 


In [23]:
stroka = "_, p, _, _ = stats.chi2_contingency(data_remote_rat_year.T.iloc[::2])"


len(stroka)



69

In [55]:
# import country_converter as coco


# ex_ussr_list = ['AM', 'AZ', 'BY', 'GE', 'KZ', 
#                 'KG', 'LV', 'LT', 'MD', 'TJ', 
#                 'TM', 'UZ', 'UA', 'EE', 'RU']

# copy_data = data.copy()
# copy_data = copy_data.reset_index()


# def continent_filling(x):
    
#     if x in ex_ussr_list:
        
#         return 'ex_ussr'
    
#     else:
#         converter = coco.CountryConverter()
#         result = converter.convert(names = x, src='ISO2', to='continent')
        
#         return result

# copy_data['company_location'] = \
#     copy_data['company_location'].apply(continent_filling)
    
# copy_data['employee_residence'] = \
#     copy_data['employee_residence'].apply(continent_filling)

# display(copy_data)

# copy_data['company_location'].nunique()
